In [2]:
import numpy as np
import cv2
import os

In [3]:
def crop(img, y, x, h, w):
    return img[y:y+h, x:x+w]

In [4]:
def save_cropped(name, img, h, w):
    center = np.array(img.shape)/2 - np.array([h, w])/2
    j = 1
    for yshift in [-1, 0, 1]:
        for xshift in [-1, 0, 1]:
            cropped_img = crop(img, center[0] + yshift*h, center[1] + xshift*w, h, w)
            cv2.imwrite(name[0:-4] + '_' + str(j) + '.tif', cropped_img)
            j += 1

In [ ]:
# create cropped grayscale images and divide them into training and testing sets
DatasetPath = '../DataSet/'
IL = [f for f in os.listdir(DatasetPath) if os.path.isfile(DatasetPath + f)]
perm = np.random.permutation(len(IL))
h = 512 # new height
w = 512 # new width
TST_DIR = DatasetPath + 'ORI_TST/'
TRN_DIR = DatasetPath + 'ORI_TRN/'
if not os.path.exists(TST_DIR):
    os.makedirs(TST_DIR)
if not os.path.exists(TRN_DIR):
    os.makedirs(TRN_DIR)

# testing set
for name in range(len(perm)/2):
    # load image in grayscale
    img = cv2.imread(DatasetPath + IL[perm[name]], 0)
    save_cropped(TST_DIR + IL[perm[name]], img, h, w)
    

# training set
for name in range(len(perm)/2, len(perm)):
    img = cv2.imread(DatasetPath + IL[perm[name]], 0)
    save_cropped(TRN_DIR + IL[perm[name]], img, h, w)

In [6]:
def GF(name, img, ws, sigma):
    cv2.imwrite(name, cv2.GaussianBlur(img, (ws, ws), sigma))

In [7]:
def JPG(name, img, quality):
    cv2.imwrite(name[:-3] + 'jpg', img, [cv2.IMWRITE_JPEG_QUALITY, quality])

In [8]:
def MF(name, img, ws):
    cv2.imwrite(name, cv2.medianBlur(img, ws))

In [9]:
def RS(name, img, size):
    cv2.imwrite(name, cv2.resize(img, (0,0), fx=size, fy=size, interpolation = cv2.INTER_CUBIC))

In [10]:
def USM(name, img, ws, sigma):
    blurred = cv2.GaussianBlur(img, (ws, ws), sigma)
    cv2.imwrite(name, 1.5*img - 0.5*blurred)

In [11]:
def WGN(name, img, std):
    AWGN = np.random.normal(scale=std, size=img.shape)
    cv2.imwrite(name, img + AWGN)

In [9]:
# create manipulated images
DatasetPath = '../DataSet/'
TST_DIR = DatasetPath + 'ORI_TST/'
TRN_DIR = DatasetPath + 'ORI_TRN/'
GFPath = DatasetPath + 'GF/'
JPGPath = DatasetPath + 'JPG/'
MFPath = DatasetPath + 'MF/'
RSPath = DatasetPath + 'RS/'
USMPath = DatasetPath + 'USM/'
WGNPath = DatasetPath + 'WGN/'
window_size = 3
sigma = 0.5
jpg_quality = 90
resize_factor = 0.8
std = 2


for ORI_DIR in [TST_DIR, TRN_DIR]:
    FLAG = ORI_DIR[-4:]
    if not os.path.exists(GFPath + FLAG):
        os.makedirs(GFPath + FLAG)
    if not os.path.exists(JPGPath + FLAG):
        os.makedirs(JPGPath + FLAG)
    if not os.path.exists(MFPath + FLAG):
        os.makedirs(MFPath + FLAG)
    if not os.path.exists(RSPath + FLAG):
        os.makedirs(RSPath + FLAG)
    if not os.path.exists(USMPath + FLAG):
        os.makedirs(USMPath + FLAG)
    if not os.path.exists(WGNPath + FLAG):
        os.makedirs(WGNPath + FLAG)
    
    IL = os.listdir(ORI_DIR)
    for name in IL:
        img = cv2.imread(ORI_DIR + name, 0)
        GF(GFPath + FLAG + name, img, window_size, sigma)
        JPG(JPGPath + FLAG + name, img, jpg_quality)
        MF(MFPath + FLAG + name, img, window_size)
        RS(RSPath + FLAG + name, img, resize_factor)
        USM(USMPath + FLAG + name, img, window_size, sigma)
        WGN(WGNPath + FLAG + name, img, std)